In [3]:
import json
from pathlib import Path

In [53]:
JSONDIR = "/tmp/reassemble/"
TSVDIR = "/tmp/word-tsv"
TXTDIR = "/Users/joregan/Playing/audio_clips/"

In [54]:
JSONPATH = Path(JSONDIR)
TSVPATH = Path(TSVDIR)
TXTPATH = Path(TXTDIR)

In [35]:
if not TSVPATH.is_dir():
    TSVPATH.mkdir()

In [55]:
def read_json(filename):
    if type(filename) is Path:
        filename = str(filename)
    out = []
    with open(filename) as inf:
        data = json.load(inf)
    for seg in data["segments"]:
        out.append({
            "start": seg["start"],
            "end": seg["end"],
            "word": seg["text"]
        })
    return out

In [64]:
def reinterpret_json(filename: Path):
    data = read_json(filename)
    stem = filename.stem
    orig_text = TXTPATH / f"{stem}.txt"
    with open(str(orig_text)) as inf:
        text = inf.read().strip()
    orig_words = text.split(" ")
    if len(orig_words) == len(data):
        for pp in zip(data, orig_words):
            if pp[0]["word"] == pp[1]:
                continue
            if pp[0]["word"] == pp[1].replace("-", ""):
                pp[0]["word"] = pp[1]
            else:
                print("Argh!", filename, pp[0]["word"], pp[1])
    return data

In [59]:
reinterpret_json(Path("/tmp/reassemble/hsi_4_0717_211_002_main_110516_112501.json"))

Maybe it's a well-known thing. Maybe it's a wellknown thing.


In [62]:
filepieces = {}
for file in JSONPATH.glob("*.json"):
    filestem = file.stem
    if not filestem.startswith("hsi_"):
        continue
    pieces = filestem.split("_")
    if len(pieces) != 8:
        continue
    base = "_".join(pieces[0:6])
    if not base in filepieces:
        filepieces[base] = []

    startint = int(pieces[6])
    endint = int(pieces[7])
    words = reinterpret_json(file)
    for word in words:
        w = word["word"].strip()
        if w.startswith("[") or w.endswith("]"):
            continue
        start = (float(startint) / 1000.0) + word["start"]
        end = (float(startint) / 1000.0) + word["end"]
        filepieces[base].append((start, end, w))

In [63]:
for piece in filepieces:
    pieces = filepieces[piece]
    pieces_sorted = sorted(pieces, key=lambda x: x[0])
    outfile = TSVPATH / f"{piece}.tsv"
    with open(outfile, "w") as of:
        for item in pieces_sorted:
            of.write(f"{item[0]:.03f}\t{item[1]:.03f}\t{item[2]}\n")